In [ ]:

! pip install opendatasets
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import opendatasets as od
import tensorflow as tf

In [ ]:
od.download("https://www.kaggle.com/datasets/dineshpiyasamara/cats-and-dogs-for-classification")


In [ ]:
BATCH_SIZE=32
IMAGE_SIZE=(128,128)

In [ ]:
train_data_dir="/content/cats-and-dogs-for-classification/cats_dogs/train"
test_data_dir="/content/cats-and-dogs-for-classification/cats_dogs/test"


In [ ]:
train_data=tf.keras.utils.image_dataset_from_directory(train_data_dir,
                                                       image_size=IMAGE_SIZE,
                                                       batch_size=BATCH_SIZE,
                                                       subset='training',
                                                       validation_split=0.2,
                                                       seed=42)


validation_data=tf.keras.utils.image_dataset_from_directory(train_data_dir,
                                                       image_size=IMAGE_SIZE,
                                                       batch_size=BATCH_SIZE,
                                                       subset='validation',
                                                       validation_split=0.1,
                                                       seed=42)


test_data=tf.keras.utils.image_dataset_from_directory(test_data_dir,
                                                       image_size=IMAGE_SIZE,
                                                       batch_size=BATCH_SIZE)

In [ ]:
class_names=train_data.class_names
class_names

In [ ]:
for image_batch,label_batch in train_data.take(1):
  print(image_batch.shape)
  print(label_batch.shape)


In [ ]:

plt.figure(figsize=(10,4))
for image,label in train_data.take(1):
  for i in range(10):
    ax=plt.subplot(2,5,i+1)
    plt.imshow(image[i].numpy().astype("uint8"))
    plt.title(class_names[label[i]])
    plt.axis("off")

In [ ]:

for image,label in train_data.take(1):
  for i in range(1):
    print(image)

In [ ]:
train_data=train_data.map(lambda x,y:(x/255,y))
validation_data=validation_data.map(lambda x,y:(x/255,y))
test_data=test_data.map(lambda x,y:(x/255,y))

In [ ]:


for image,label in train_data.take(1):
  for i in range(1):
    print(image)

In [ ]:


data_augmentation=tf.keras.Sequential([
    tf.keras.layers.RandomFlip("horizontal",input_shape=(128,128,3)),
    tf.keras.layers.RandomRotation(0.2),
    tf.keras.layers.RandomZoom(0.2),
])

In [ ]:
model=tf.keras.models.Sequential()

model.add(data_augmentation)

model.add(tf.keras.layers.Conv2D(32,kernel_size=3,activation="relu"))
model.add(tf.keras.layers.MaxPooling2D())

model.add(tf.keras.layers.Conv2D(64,kernel_size=3,activation="relu"))
model.add(tf.keras.layers.MaxPooling2D())

model.add(tf.keras.layers.Conv2D(128,kernel_size=3,activation="relu"))
model.add(tf.keras.layers.MaxPooling2D())

model.add(tf.keras.layers.Dropout(0.2))
model.add(tf.keras.layers.BatchNormalization())

model.add(tf.keras.layers.Flatten())

model.add(tf.keras.layers.Dense(128,activation="relu"))
model.add(tf.keras.layers.Dense(128,activation="relu"))
model.add(tf.keras.layers.Dense(32,activation="relu"))

model.add(tf.keras.layers.Dense(1,activation="sigmoid"))

In [ ]:
model.summary


In [ ]:

model.compile(optimizer=tf.keras.optimizers.Adam(),
              loss=tf.keras.losses.BinaryCrossentropy(),
              metrics=["accuracy"])

In [ ]:

history=model.fit(train_data,
                  epochs=20,
                  validation_data=validation_data)

In [ ]:

fig = plt.figure()
plt.plot(history.history['accuracy'], color='teal', label='accuracy')
plt.plot(history.history['val_accuracy'], color='orange', label='val_accuracy')
fig.suptitle('Accuracy', fontsize=20)
plt.legend()
plt.show()

In [ ]:
history.history

In [ ]:
precision=tf.keras.metrics.Precision()
recall=tf.keras.metrics.Recall()
accuracy=tf.keras.metrics.BinaryAccuracy()

In [ ]:

fig=plt.figure()
plt.plot(history.history['loss'],color='teal',label='loss')
plt.plot(history.history['val_loss'],color='orange',label='val_loss')
fig.suptitle('Loss',fontsize=20)
plt.legend(loc='upper left')
plt.show()

In [ ]:

for batch in test_data.as_numpy_iterator():
  X,y=batch
  yhat=model.predict(X)
  precision.update_state(y,yhat)
  recall.update_state(y,yhat)
  accuracy.update_state(y,yhat)

In [ ]:

precision.result()

In [ ]:

recall.result()

In [ ]:

accuracy.result()

In [ ]:

!pip install opencv-python

In [ ]:
import cv2
import matplotlib.pyplot as plt

image = cv2.imread("/content/cats-and-dogs-for-classification/cats_dogs/test/cats/cat.4005.jpg")
plt.imshow(image)
plt.show()

In [ ]:
resized_image = tf.image.resize(image, IMAGE_SIZE)
scaled_image = resized_image / 255

In [ ]:

scaled_image

In [ ]:

np.expand_dims(scaled_image,0).shape

In [ ]:

y_hat=model.predict(np.expand_dims(scaled_image,0))

In [ ]:

y_hat

In [ ]:
class_names

In [ ]:
if y_hat>0.5:
  print(class_names[1])
else:
  print(class_names[0])